In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/intern_icomm_project_demo/'
! ls

/content/drive/My Drive/intern_icomm_project_demo
data		  K_mean_nlp.ipynb	   vietnamese-stopwords.txt
data_all_nlp.csv  linear_regression.ipynb  x28.txt
data.zip	  log.csv


In [ ]:
!wget https://people.sc.fsu.edu/~jburkardt/datasets/regression/x28.txt

--2020-09-04 13:48:22--  https://people.sc.fsu.edu/~jburkardt/datasets/regression/x28.txt
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.16.102
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.16.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7952 (7.8K) [text/plain]
Saving to: ‘x28.txt.1’

x28.txt.1           100%[===================>]   7.77K  --.-KB/s    in 0.001s  

2020-09-04 13:48:24 (9.16 MB/s) - ‘x28.txt.1’ saved [7952/7952]



In [ ]:
import pandas as pd
import csv
import numpy as np

In [ ]:
# with open('x28.txt', 'r') as infile, open('log.csv', 'w') as outfile:
#     for line in infile.readlines():
#         outfile.write(" ".join(line.split()).replace(' ', ','))
#         outfile.write("\n") # trailing comma shouldn't matter

In [ ]:
def get_data(path_input, path_output):
    with open(path_input, 'r') as infile, open(path_output, 'w') as outfile:
        for line in infile.readlines():
            outfile.write(" ".join(line.split()).replace(' ', ','))
            outfile.write("\n") # trailing comma shouldn't matter
    data = pd.read_csv(path_output, header=None)
    column_name = ['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'B']
    data = pd.read_csv(path_output, names=column_name)
    data = data.drop('A0', axis=1)
    return data

In [ ]:
def caculator_min_max(df):
    max = df.max(axis = 0, skipna = True) 
    min = df.min(axis = 0, skipna = True) 
    return max, min

In [ ]:
def normaline_data(col, max, min):
    col_norm = []
    for x in col:
        s= (x-min)/(max-min)
        col_norm.append(s)
    return col_norm

In [ ]:
def process_normaline_data():
    data = get_data('x28.txt', 'log.csv')
    i = 0
    df = pd.DataFrame()
    X0 = pd.DataFrame()
    max, min = caculator_min_max(data)
    for col in data.columns:
        df[col] = normaline_data(data[col], max[i], min[i])
        i += 1
    X0['X0'] = [1]*df.shape[0]
    X1 = df.drop('B', axis=1)
    X = X0.join(X1)
    y = df['B']
    return X, y

In [ ]:
X.shape, y.shape

((60, 16), (60,))

In [ ]:
type(y)

In [ ]:
# x = np.hstack((np.ones((X.shape[0], 1)), X))
# print(x.shape)
# w = 1/x.shape[1]*np.ones((x.shape[1], 1))
# print(w.shape)

# y = y.reshape(-1,1)

In [ ]:
len(np.dot(x, w))

60

In [ ]:
# numOfIteration = 100
# cost = np.zeros((numOfIteration,1))
# learning_rate = 0.0000001
# for j in range(1, numOfIteration):
#     r = np.dot(x, w) - y.reshape
#     cost[j] = 0.5*np.sum(r*r)
#     w[0] -= learning_rate*np.sum(r)*1/x.shape[0]
#     for i in range(1,x.shape[1]):
#         w[i] -= 1/x.shape[0]*learning_rate*np.sum(np.multiply(r, x[:,i].reshape(-1,1)))
#         print(str(i) +":"+ str(w[i]))
#     print(cost[j])

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
class RidgeRegression():
    def __init__(self):
        return
    def fit(self, X, y, lamda):
        W = np.linalg.inv((X.transpose().dot(X))-lamda*np.identity(X.shape[1])).dot(X.transpose()).dot(y)
        # print('W', W.shape)
        return W
    def predict(self, W, X_val):
        X_val = np.array(X_val)
        # print('X_val', X_val.shape)
        y_pred = X_val.dot(W)
        return y_pred
    def compute_RSS(self, y, y_pred):
        # print("y:", y.shape)
        # print("y_pred:", y_pred.shape)
        loss = (1.0/y.shape[0])*np.sum((y-y_pred)**2)
        return loss
    def get_the_best_lamda(self, X, y):
        def kfold(n_folds, lamda):
            avg_loss = 0   
            k_fold = KFold(n_splits = n_folds, shuffle=False)
            for train_index, val_index in k_fold.split(X):
                X_train, X_val = X.iloc[list(train_index)], X.iloc[list(val_index)]
                y_train, y_val = y.iloc[list(train_index)], y.iloc[list(val_index)]
                W = self.fit(X_train, y_train, lamda)
                y_pred = self.predict(W, X_val)
                avg_loss += self.compute_RSS(y_val, y_pred)
            return avg_loss/n_folds
        def range_scan(best_lamda, minimum_RSS, lamda_value):
            for current_lamda in lamda_value:
                avg_RSS_kfold = kfold(n_folds = 5, lamda = current_lamda)
                if avg_RSS_kfold < minimum_RSS:
                    minimum_RSS = avg_RSS_kfold
                    best_lamda = current_lamda
            return best_lamda, minimum_RSS
        best_lamda, minimum_RSS = range_scan(best_lamda=0, minimum_RSS=1000**2, lamda_value=range(30))
        lamda_value = [k*1.0/1000 for k in range((max(0, (best_lamda-1)*1000)), (best_lamda+1)*1000, 1)]
        best_lamda, minimum_RSS = range_scan(best_lamda=best_lamda, minimum_RSS=minimum_RSS, lamda_value=lamda_value)
        return best_lamda                                


In [ ]:
X, y = process_normaline_data()

In [ ]:
ride_regression = RidgeRegression()

In [ ]:
best_lamda = ride_regression.get_the_best_lamda(X, y)
print(best_lamda)

0


In [ ]:
X_train, X_test = X[:48], X[48:]
y_train, y_test = y[:48], y[48:]

In [ ]:
best_lamda = ride_regression.get_the_best_lamda(X_train, y_train)
print(best_lamda)

0


In [ ]:
W_new = ride_regression.fit(X_train, y_train, best_lamda)

In [ ]:
W_new

array([ 0.90555204,  0.34131646, -0.66371637, -0.18594276, -0.44045134,
       -0.37467461, -0.28077816, -0.06542953,  0.33090887,  0.34896807,
        0.08045835,  0.12287046, -2.29373543,  2.36221282, -0.07828573,
        0.05089616])

In [ ]:
y_pred = ride_regression.predict(W_new, X_test)

In [ ]:
y_pred

array([-0.13123028,  0.36811492,  0.26348479,  0.38026105,  0.44964155,
        0.30332102,  0.6081895 ,  0.32621614,  0.43128838,  0.2924141 ,
        0.3169103 ,  0.39277089])

In [ ]:
def de_normaline_data(col, max, min):
    col_norm = []
    for x in col:
        s= x*(max-min)+min
        col_norm.append(s)
    return col_norm

In [ ]:
column_name = ['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'B']
data = pd.read_csv('log.csv', names=column_name)
data = data.drop('A0', axis=1)

In [ ]:
max, min = caculator_min_max(data)

In [ ]:
y_pred_denorm = de_normaline_data(y_pred, max['B'], min['B'])

In [ ]:
y_pred_denorm

[748.4213395684873,
 909.4217175029173,
 875.6865573093067,
 913.3379077902866,
 935.7077783246748,
 888.5306724852093,
 986.8272822651254,
 895.9125863563696,
 929.790292016399,
 885.0140301468696,
 892.9121680705539,
 917.3713673795567]

In [ ]:
y_test_denorm = de_normaline_data(y_test, max['B'], min['B'])

In [ ]:
y_test_denorm

[790.7330000000001,
 899.2639999999999,
 904.155,
 950.672,
 972.4639999999999,
 912.202,
 967.803,
 823.7639999999999,
 1003.502,
 895.696,
 911.817,
 954.442]

In [ ]:
loss = ride_regression.compute_RSS(y_test, y_pred)

In [ ]:
loss

0.015114947329812648

use libary

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
y_pred

array([-0.13123028,  0.36811492,  0.26348479,  0.38026105,  0.44964155,
        0.30332102,  0.6081895 ,  0.32621614,  0.43128838,  0.2924141 ,
        0.3169103 ,  0.39277089])

In [ ]:
score = reg.score(X_test, y_test)